# 1. Configurar ambiente

In [ ]:
#Instalando bibliotecas necesarias
import pandas as pd
import numpy as np
import re
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
nlp = spacy.load('es_core_news_md')
!pip install jellyfish
import jellyfish
import requests
!pip install python-docx
import csv
from docx import Document
import nltk
nltk.download('punkt')
global diccionario_irregulares, documento, lista_frases, lista_frases_normalizadas
diccionario_irregulares, documento = {}, ''
import warnings, os
warnings.filterwarnings('ignore')

#2. Tratamiento de datos

In [190]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  radio=0
  palabra_encontrada=palabra
  for word in lista_verbos:
    confianza = jellyfish.jaro_winkler(palabra, word)
    if (confianza>=0.93 and confianza>=radio):
      radio=confianza
      palabra_encontrada=word
  return palabra_encontrada

#Función para tratar los textos
def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = " ".join(texto.split())
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
  patron = r"(es|me|as|te|ste)$"
  nueva_palabra = re.sub(patron, "r", palabra)
  return nueva_palabra.split()[0]

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  if len(tokens)==0:
    if [x for x in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo'] if x in tratamiento_texto(texto)]: tokens.append('elprofealejo')
    elif [x for x in ['cientifico de datos', 'data scientist'] if x in tratamiento_texto(texto)]: tokens.append('datascientist')
    elif [x for x in ['ciencia de datos', 'data science'] if x in tratamiento_texto(texto)]: tokens.append('datascience')
    elif [x for x in ['big data', 'bigdata'] if x in tratamiento_texto(texto)]: tokens.append('bigdata')
  else:
    elementos_a_eliminar = ["profe", "alejo", "profealejo", "cual", "que", "quien", "cuanto", "donde", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: tokens.remove('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: tokens.remove('contar') 
    elif 'hago' in texto and 'hacer' in tokens: tokens.remove('hacer') 
    elif 'entiendes' in texto and 'entender' in tokens: tokens.remove('entender') 
    elif 'sabes' in texto and 'saber' in tokens: tokens.remove('saber')
    tokens = [x.replace('datar','data').replace('datos','dato') for x in tokens if x not in elementos_a_eliminar]
  return tokens

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  tokens=revisar_tokens(texto, tokens)
  if 'elprofealejo' in tokens:
    texto = ' '.join(texto.split()[:15])
  else:
    texto = ' '.join(texto.split()[:25])

  doc = nlp(texto)
  for t in doc:
    lemma=diccionario_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  tokens = revisar_tokens(texto, tokens)
  return tokens

# 3. Cargar bases de verbos

In [3]:
#Importando verbos en español
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
trans = str.maketrans('áéíóú','aeiou')
lista_verbos=[]
url = ['https://www.ejemplos.co/verbos-mas-usados-en-espanol/',
       'https://www.ejemplos.co/verbos-predicativos/',
       'https://www.ejemplos.co/verbos-personales/',
       'https://www.ejemplos.co/verbos-irregulares/',
       'https://www.ejemplos.co/verbos/',
       'https://www.ejemplos.co/100-ejemplos-de-verbos-regulares/',
       'https://www.ejemplos.co/verbos-del-decir/',
       'https://www.ejemplos.co/verbos-con-a/',
       'https://www.ejemplos.co/verbos-con-b/',
       'https://www.ejemplos.co/verbos-con-c/',
       'https://www.ejemplos.co/verbos-con-d/',
       'https://www.ejemplos.co/verbos-con-e/',
       'https://www.ejemplos.co/verbos-con-f/',
       'https://www.ejemplos.co/verbos-con-g/',
       'https://www.ejemplos.co/verbos-con-h/',
       'https://www.ejemplos.co/verbos-con-i/',
       'https://www.ejemplos.co/verbos-con-j/',
       'https://www.ejemplos.co/verbos-con-k/',
       'https://www.ejemplos.co/verbos-con-l/',
       'https://www.ejemplos.co/verbos-con-ll/',
       'https://www.ejemplos.co/verbos-con-m/',
       'https://www.ejemplos.co/verbos-con-n/',
       'https://www.ejemplos.co/verbos-con-o/',
       'https://www.ejemplos.co/verbos-con-p/',
       'https://www.ejemplos.co/verbos-con-q/',
       'https://www.ejemplos.co/verbos-con-r/',
       'https://www.ejemplos.co/verbos-con-s/',
       'https://www.ejemplos.co/verbos-con-t/',
       'https://www.ejemplos.co/verbos-con-u/',
       'https://www.ejemplos.co/verbos-con-v/',
       'https://www.ejemplos.co/verbos-con-w/',
       'https://www.ejemplos.co/verbos-con-x/',
       'https://www.ejemplos.co/verbos-con-y/',
       'https://www.ejemplos.co/verbos-con-z/']

for i in range(len(url)):
  try:
    respuesta = requests.get(url[i], headers=headers)
    respuesta = respuesta.content.decode('utf-8')
    bases = pd.read_html(respuesta)
    for i, df in enumerate(bases):
      for idx,row in bases[i].iterrows():     
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[0].split('/')]
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[1].split('/')]
          _ = [lista_verbos.append(re.sub(r"\((.*?)\)", '', x.lower()).strip().translate(trans)) for x in row[2].split('/')]
  except Exception:
    continue
lista_verbos = [elemento for elemento in lista_verbos if (elemento=='ir' or len(elemento)!=2)]
nuevos_verbos = ['costar', 'referir', 'datar']
lista_verbos.extend(nuevos_verbos)
lista_verbos=list(set(lista_verbos))

# Definir una lista de verbos irregulares y sus conjugaciones en pasado, presente, futuro, imperfecto, pretérito y condicional
verbos_irregulares = [
    ('ser', 'soy', 'eres', 'seras', 'eras', 'es', 'serias'),
    ('estar', 'estuviste', 'estas', 'estaras', 'estabas', 'estuviste', 'estarias'),
    ('ir', 'fuiste', 'vas', 'iras', 'ibas', 'fuiste', 'irias'),
    ('ir', 'fuiste', 'vaya', 'iras', 'ibas', 'fuiste', 'irias'),
    ('tener', 'tuviste', 'tienes', 'tendras', 'tenias', 'tuviste', 'tendrias'),
    ('hacer', 'hiciste', 'haces', 'haras', 'hacias', 'hiciste', 'harias'),
    ('decir', 'dijiste', 'dices', 'diras', 'decias', 'dijiste', 'dirias'),
    ('decir', 'dimar', 'dime', 'digame', 'dimir', 'dimo', 'dimiria'),
    ('poder', 'pudiste', 'puedes', 'podras', 'podias', 'pudiste', 'podrias'),
    ('saber', 'supiste', 'sabes', 'sabras', 'sabias', 'supiste', 'sabrias'),
    ('poner', 'pusiste', 'pones', 'pondras', 'ponias', 'pusiste', 'pondrias'),
    ('ver', 'viste', 'ves', 'veras', 'veias', 'viste', 'verias'),
    ('dar', 'diste', 'das', 'daras', 'dabas', 'diste', 'darias'),
    ('dar', 'damar', 'dame', 'daras', 'dabas', 'darme', 'darias'),
    ('venir', 'viniste', 'vienes', 'vendras', 'venias', 'viniste', 'vendrias'),
    ('haber', 'haya', 'has', 'habras', 'habias', 'hubiste', 'habrias'),
    ('caber', 'cupiste', 'cabes', 'cabras', 'cabias', 'cupiste', 'cabrias'),
    ('valer', 'valiste', 'vales', 'valdras', 'valias', 'valiste', 'valdrias'),
    ('querer', 'quisiste', 'quieres', 'querras', 'querias', 'quisiste', 'querrias'),
    ('llegar', 'llegaste', 'llegares', 'llegaras', 'llegarias', 'llegaste', 'llegarrias'),
    ('hacer', 'hiciste', 'haces', 'haras', 'hacias', 'hiciste', 'harias'),
    ('decir', 'dijiste', 'dices', 'diras', 'decias', 'dijiste', 'dirias'),
    ('poder', 'pudiste', 'puedes', 'podras', 'podias', 'pudiste', 'podria'),
    ('contar', 'contaste', 'cuentas', 'contaras', 'contabas', 'cuentame', 'contarias'),
    ('saber', 'supiste', 'sabes', 'sabras', 'sabias', 'supiste', 'sabrias'),
    ('costar', 'cuesta', 'cuestan', 'costo', 'costaria', 'costarian', 'cuestas'),
    ('durar', 'duraste', 'duro', 'duraras', 'durabas', 'duraste', 'durarias')
]

# Crear el DataFrame
df = pd.DataFrame(verbos_irregulares, columns=['Verbo', 'Pasado', 'Presente', 'Futuro', 'Imperfecto', 'Pretérito', 'Condicional'])
for columna in df.columns:
  if columna != 'Verbo':
    for valor in df[columna]:
      diccionario_irregulares[valor] = df.loc[df[columna] == valor, 'Verbo'].values[0]

# 4. Cargar bases de documentos

In [144]:
#Importando bases csv
txt_folder_path = '/content/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for idx in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[idx], "r", encoding="utf-8") as archivo_csv:
    lector_csv = csv.reader(archivo_csv)
    for fila in lector_csv:
      if fila[-1]!='frase':
        documento_csv += fila[-1]
documento_csv

"El curso 'Fundamentos de Ciencia de Datos' tiene un tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023. El curso 'Análisis Exploratorio de Datos' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023. El curso 'Aprendizaje Automático en Ciencia de Datos' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de septiembre de 2023. El curso 'Fundamentos de Machine Learning' tiene tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023. El curso 'Aprendizaje Supervisado y No Supervisado' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023. El curso 'Redes Neuronales y Deep Learning' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de septiembre de 

In [145]:
#Importando bases docx
txt_folder_path = '/content/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for idx in range(len(lista_documentos)):
  for paragraph in Document(txt_folder_path+'/'+lista_documentos[idx]).paragraphs:
    documento_docx += paragraph.text.replace('*','\n\n*').replace('-','\n-')
documento_docx

'ElProfeAlejo es un consultor especializado en Ciencia de Datos con una amplia experiencia en diversos ámbitos, incluyendo Business Intelligence, Big Data, Data Science, Cloud Computing y Marketing Digital. ElProfeAlejo nació en Lima, Perú, actualmente trabaja en São Paulo, Brasil. Con más de 20 años de experiencia en varios países, ElProfeAlejo ha trabajado en proyectos internacionales relacionados con Business Intelligence, Big Data e Innovación Tecnológica, ha interactuado diariamente con consultores, gestores y directores de diversas áreas de Telefónica Brazil y Telefónica Global, lo que le ha permitido adquirir un amplio conocimiento y experiencia en el campo. ElProfeAlejo posee un conocimiento sólido de diversos lenguajes, herramientas y softwares utilizados en Ciencia de Datos, como SAS, Teradata, SQL Server, R, Python, Spark, Hive, Machine Learning, Tableau, MicroStrategy, Power BI, PowerPivot, entre otros, su capacidad analítica es predominante y se destaca por su excelente co

In [146]:
#Importando bases txt
txt_folder_path = '/content/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for idx in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[idx], "r", encoding="utf-8") as archivo_txt:
    lector_txt = archivo_txt.read()
    for fila in lector_txt:
      documento_txt += fila
documento_txt

'La ciencia de datos es un campo académico interdisciplinario que utiliza estadística, computación científica, métodos, procesos, algoritmos y sistemas científicos para obtener (recolectar o extraer), tratar, analizar y presentar informes a partir de datos ruidosos, estructurados y no estructurados. \n\nLa ciencia de datos es multifacética y puede describirse como una ciencia, un paradigma de investigación, un método de investigación, una disciplina, un flujo de trabajo o una profesión. \n\nLa ciencia de datos integra el conocimiento del dominio de la aplicación subyacente (por ejemplo, ciencia económica, finanzas, medicina, ciencias naturales, tecnologías de la información) y es un "concepto consistente en unificar estadística, análisis de datos, informática y sus métodos relacionados" para "comprender y analizar fenómenos reales" con datos. \n\nLa ciencia de datos utiliza métodos, técnicas y teorías extraídas de muchos campos dentro del contexto de las matemáticas, las estadísticas, 

In [147]:
documento = documento_csv + documento_txt + documento_docx
documento

'El curso \'Fundamentos de Ciencia de Datos\' tiene un tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023. El curso \'Análisis Exploratorio de Datos\' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023. El curso \'Aprendizaje Automático en Ciencia de Datos\' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de septiembre de 2023. El curso \'Fundamentos de Machine Learning\' tiene tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023. El curso \'Aprendizaje Supervisado y No Supervisado\' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023. El curso \'Redes Neuronales y Deep Learning\' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de se

In [148]:
lista_frases = nltk.sent_tokenize(documento,'spanish')
lista_frases_normalizadas = [' '.join(normalizar(x)) for x in lista_frases]
lista_frases_normalizadas

['datascience curso fundamentos ciencia dato tener tiempo duracion 8 semana precio costar 500 usd',
 'curso analisis exploratorio dato tener tiempo duracion 6 semana precio costar 400 usd fecha',
 'datascience curso aprendizaje automatico ciencia dato tener tiempo duracion 10 semana precio costar 700 usd',
 'curso fundamentos machine learning tener tiempo duracion 8 semana precio costar 500 usd fecha',
 'curso aprendizaje supervisado no tener tiempo duracion 6 semana precio costar 400 usd fecha',
 'curso redes neuronales deep learning tener tiempo duracion 10 semana precio costar 700 usd fecha',
 'curso introduccion inteligencia artificial tener tiempo duracion 8 semana precio costar 500 usd fecha',
 'curso aplicaciones practicas inteligencia artificial tener tiempo duracion 6 semana precio costar 400 usd',
 'curso etica responsabilidad inteligencia artificial tener tiempo duracion 4 semana precio costar 300 usd',
 'datascience ciencia dato ser campo academico interdisciplinario utiliz

# 5. Buscar respuesta del Chatbot

In [206]:
#Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    #print(frase, ' :',sum(1 for elemento in pregunta if elemento in frase))
    return sum(1 for elemento in pregunta if elemento in frase) 

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:6]
  if 'curso' not in pregunta: lista = [frase for frase in lista if 'curso' not in frase]
  lista.append(' '.join(pregunta))
  #for elemento in lista: print(elemento)
  TfidfVec = TfidfVectorizer(tokenizer=normalizar)
  tfidf = TfidfVec.fit_transform(lista)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  return req_tfidf, lista_frases[diccionario[lista[idx]]]

# 5. Ejecutar Chatbot

In [213]:
pregunta='que es la vida?'
print('Tokens Pregunta: ', normalizar(pregunta))
respuesta = respuesta_documento(pregunta)
print('Tokens Respuesta: ', normalizar(respuesta[1]))
print('Precisión: ', round(respuesta[0],2))
print(respuesta[1])

Tokens Pregunta:  ['ser', 'vida']
Tokens Respuesta:  ['datascience', 'ciencia', 'dato', 'ser', 'multifacetico', 'poder', 'describir', 'paradigma', 'investigacion', 'metodo', 'disciplina', 'flujo']
Precisión:  0.05
La ciencia de datos es multifacética y puede describirse como una ciencia, un paradigma de investigación, un método de investigación, una disciplina, un flujo de trabajo o una profesión.


In [210]:
lista_frases = nltk.sent_tokenize(documento,'spanish')
TfidfVec = TfidfVectorizer(tokenizer=normalizar)
tfidf = TfidfVec.fit_transform(lista_frases)

# Obtén los tokens
tokens = TfidfVec.get_feature_names_out()

# Imprime los tokens para cada frase
for i, frase in enumerate(lista_frases):
  tokens_frase = tfidf[i].nonzero()[1]  # Índices de los tokens presentes en la frase
  elementos_tokenizados = [tokens[token_idx] for token_idx in tokens_frase]
  print(f"{lista_frases[i]}")
  print(elementos_tokenizados)
  print()

El curso 'Fundamentos de Ciencia de Datos' tiene un tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023.
['usd', '500', 'costar', 'precio', 'semana', '8', 'duracion', 'tiempo', 'tener', 'dato', 'ciencia', 'fundamento', 'curso', 'datascience']

El curso 'Análisis Exploratorio de Datos' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023.
['fecha', '400', '6', 'exploratorio', 'analisis', 'usd', 'costar', 'precio', 'semana', 'duracion', 'tiempo', 'tener', 'dato', 'curso']

El curso 'Aprendizaje Automático en Ciencia de Datos' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de septiembre de 2023.
['700', '10', 'automatico', 'aprendizaje', 'usd', 'costar', 'precio', 'semana', 'duracion', 'tiempo', 'tener', 'dato', 'ciencia', 'curso', 'datascience']

El curso 'Fundamentos de Machine Learning' tiene tiempo de duració